# Project Ruby

Customer survey analysis

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
image_directory="images/"
image_extension="png"

def save_figure(fig, filename):
    filepath = image_directory + filename + "." + image_extension
    fig.savefig(filepath)

In [4]:
NORMAL_SIZE = 12
#BIGGER_SIZE = 14
#BIGGEST_SIZE = 16

plt.rc('font', size=NORMAL_SIZE)          # controls default text sizes
#plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
#plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=NORMAL_SIZE)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=NORMAL_SIZE)    # fontsize of the tick labels
#plt.rc('legend', fontsize=NORMAL_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=BIGGEST_SIZE)  # fontsize of the figure title

sns.set(font_scale=1.1)

# Data Preparation

In [ ]:
bank_data = pd.read_csv("r.csv");
display(bank_data.shape) # rows & columns